In [3]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

C:\Users\1686110\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
!pip install gensim

In [35]:
data = ["I love machine learning. Its awesome.",
        "I love coding in python",
        "I love building chatbots",
        "they chat amagingly well"]

In [5]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

In [6]:
tagged_data

[TaggedDocument(words=['i', 'love', 'machine', 'learning', '.', 'its', 'awesome', '.'], tags=['0']),
 TaggedDocument(words=['i', 'love', 'coding', 'in', 'python'], tags=['1']),
 TaggedDocument(words=['i', 'love', 'building', 'chatbots'], tags=['2']),
 TaggedDocument(words=['they', 'chat', 'amagingly', 'well'], tags=['3'])]

In [11]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

In [17]:
model.save("d2v.model")
# [TaggedDocument(words=['i', 'love', 'machine', 'learning', '.', 'its', 'awesome', '.'], tags=['0']),
#  TaggedDocument(words=['i', 'love', 'coding', 'in', 'python'], tags=['1']),
#  TaggedDocument(words=['i', 'love', 'building', 'chatbots'], tags=['2']),
#  TaggedDocument(words=['they', 'chat', 'amagingly', 'well'], tags=['3'])]

In [43]:
from gensim.models.doc2vec import Doc2Vec

# model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("I love building".lower())
v1 = model.infer_vector(test_data)
print("**************************************************")
print("V1_infer", v1)
print("***************************************************")
# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('1')
print("########################################")
print(similar_doc)
print("####################################")

# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
print(model.docvecs['1'])

**************************************************
V1_infer [ 0.00649048 -0.01596985 -0.01124553 -0.00185211  0.00531147  0.00670751
 -0.00380727 -0.02258476 -0.02615076 -0.0206571  -0.02128661 -0.02280711
 -0.01100331 -0.02008787 -0.0160804   0.02307077 -0.0245235  -0.01585891
  0.01246411 -0.02122281]
***************************************************
########################################
[('2', 0.912226676940918), ('0', 0.7618046998977661), ('3', 0.7178292870521545)]
####################################
[-0.44645947  0.05181074 -0.46369132  0.07062578  0.60032445 -0.6622833
 -0.625254   -0.6537094   0.07582696 -0.7323618   0.29516125  0.50264066
 -0.7469842  -0.7266871  -0.28869122  0.07489765  0.00168621 -0.3514909
 -0.65987617  0.21526825]


<ipython-input-43-f9e7a7655d1e>:11: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar('1')
<ipython-input-43-f9e7a7655d1e>:17: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  print(model.docvecs['1'])


In [36]:
import pandas as pd
# from bert_serving.client import BertClient
from elasticsearch import Elasticsearch

In [37]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
model= Doc2Vec.load("d2v.model")

In [21]:
def create_index():
    request_body = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },

        'mappings': {
                'properties': {
                    'ticket': {
                        'type': 'text'
                    },
                    'ticket_vector': {
                            "type": "dense_vector",
                            "dims": 20
                          }
                }}
    }
    print("creating 'qna_index' index...")
    create_index = es.indices.create(index="check_tickets", body=request_body)
    print("Index-",create_index)


In [25]:
a = create_index()

creating 'qna_index' index...
Index- {'acknowledged': True, 'shards_acknowledged': True, 'index': 'check_tickets'}


In [26]:
def Create_Body(total, ticket_vector):
    print("creating body")
    try:
        RecordBody = {
            "ticket": total ,
            "ticket_vector": ticket_vector,
             }

        return RecordBody
    except:
        print("Body for this record cannot be created.")

In [27]:
def Index_Record(RecordBody):
    output = es.index(index='check_tickets', body=RecordBody)
    if (output['_id']):
        return 1

In [30]:
def insert_into_es():
    print("inside dunc")
    total = ["I love machine learning. Its awesome.",
        "I love coding in python",
        "I love building chatbots",
        "they chat amagingly well"]
    for i in total:
        print(i)
        test_data = word_tokenize(i.lower())
        ticket_vector = model.infer_vector(test_data)
        print(ticket_vector)
#         ticket_vector = ticket_vector[0].tolist()
        print(ticket_vector)
        record_body = Create_Body(i, ticket_vector)
        print(record_body)
        count = Index_Record(record_body)
    print("DDDDDDDDDDDDDDDDDDDDDOOOOOOOOOOOOOONNNNEEEEEEEEEEEE")

In [31]:
c = insert_into_es()

inside dunc
I love machine learning. Its awesome.
[-0.00615108  0.00183774 -0.02271492  0.00696359  0.03945305 -0.0244226
 -0.01748043 -0.00384978  0.00428472 -0.03002976 -0.01226205  0.03000083
 -0.02332551 -0.05103941 -0.02740873  0.0142898  -0.02631849 -0.00370299
 -0.0294555   0.02768894]
[-0.00615108  0.00183774 -0.02271492  0.00696359  0.03945305 -0.0244226
 -0.01748043 -0.00384978  0.00428472 -0.03002976 -0.01226205  0.03000083
 -0.02332551 -0.05103941 -0.02740873  0.0142898  -0.02631849 -0.00370299
 -0.0294555   0.02768894]
creating body
{'ticket': 'I love machine learning. Its awesome.', 'ticket_vector': array([-0.00615108,  0.00183774, -0.02271492,  0.00696359,  0.03945305,
       -0.0244226 , -0.01748043, -0.00384978,  0.00428472, -0.03002976,
       -0.01226205,  0.03000083, -0.02332551, -0.05103941, -0.02740873,
        0.0142898 , -0.02631849, -0.00370299, -0.0294555 ,  0.02768894],
      dtype=float32)}
I love coding in python
[-0.02880793 -0.01530779  0.00467234 -0.0261

In [ ]:
# test_data = word_tokenize("I love chatbots".lower())
# v1 = model.infer_vector(test_data)

In [5]:
model =  Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [6]:
def query_index(new_question):
    test_data = word_tokenize(new_question.lower())
    query_vector  =  model.infer_vector(test_data)
#    query_vector = query_vector[0].tolist()
    #print("LEN-",len(query_vector))
    num_outputs=3
    query = {
    "script_score": {
        "query": {"match_all": {}},
        "script": {
        "source": "cosineSimilarity(params.query_vector, 'ticket_vector') +1.0",
        "params": {"query_vector": query_vector}
        }
    }
    }

    body = {
        "size": num_outputs,
        "query": query,
    }

    output = es.search(body=body,index="check_tickets")
    # print(output)
    if len(output['hits']['hits']) >= 4:
        iter_len = 4
    else:
        iter_len = len(output['hits']['hits'])
    for i in range(iter_len):
        
        tmp = output['hits']['hits'][i]
        print("#"*100)
        print("I-",i)
        #print("Top -",(i+1),": Score -",output['hits']['hits'][i]['_score'])
        #print("question-",(tmp['_source']['question']))

        question = (tmp['_source']['ticket'])
        print(question)


In [7]:
query_index("I love coding in python")

AttributeError: 'Doc2Vec' object has no attribute 'syn1neg'